<a href="https://colab.research.google.com/github/Srinjana/Steganos/blob/main/audio_steganography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import os
import wave
import librosa
import soundfile as sf

In [27]:
def house_keeping():
  os.remove("tmp.wav")
  print("[INFO] Cleanup complete")

In [28]:
def encoding (file_nm, string):

    # Read frames and convert to byte array
    x,_ = librosa.load(file_nm, sr=16000)
    sf.write('tmp.wav', x, 16000)
    song = wave.open('tmp.wav', mode='rb')
    frame_bytes = bytearray(list(song.readframes(song.getnframes())))

    # Append dummy data to fill out rest of the bytes. Receiver shall detect and remove these characters.
    string = string + int((len(frame_bytes)-(len(string)*8*8))/8) * '#'
    # Convert text to bit array
    bits = list(map(int, ''.join([bin(ord(i)).lstrip('0b').rjust(8, '0') for i in string])))

    # Replacing LSB of each byte of the audio data by one bit from the text bit array
    for i, bit in enumerate(bits):
        frame_bytes[i] = (frame_bytes[i] & 254) | bit
    # Get the modified bytes
    frame_modified = bytes(frame_bytes)

    # Write bytes to a new wave audio file
    with wave.open('encoded.wav', 'wb') as fd:
        fd.setparams(song.getparams())
        fd.writeframes(frame_modified)
    song.close()
    house_keeping()


In [29]:
def decoding(en_file_nm):
    song = wave.open(en_file_nm, mode='rb')


    # Convert audio to byte array
    frame_bytes = bytearray(list(song.readframes(song.getnframes())))

    # Extract the LSB of each byte
    extracted = [frame_bytes[i] & 1 for i in range(len(frame_bytes))]
    # Convert byte array back to string
    string = "".join(chr(
        int("".join(map(str, extracted[i:i+8])), 2)) for i in range(0, len(extracted), 8))
    # Cut off at the filler characters
    decoded = string.split("###")[0]

    return decoded
    song.close()


In [30]:
# Driver Code

if __name__ == "__main__":
    while True:
        print("  ")
        print("--- SELECT AN OPTION TO PERFORM AUDIO STEGANOGRAPHY ---")
        print("1.Hide a secret message in an audio file \n2.Reveal the secret message from the audio")
        print("Enter any other key to exit")
        choice = input("Enter your key here: ")
        if (choice == '1'):
            file_nm = input("Enter the name of your audio-file of choice, with extension: ")
            secret_msg = input("Enter your secret message: ")

            encoding(file_nm, secret_msg)
            print("Encoding...")
            print("[INFO] YOUR MESSAGE HAS BEEN RECORDED.")
            print("[INFO] Your encoded audio is saved as 'encoded.wav'.")

        elif (choice == '2'):
            en_file_nm = input("Enter the name of your encoded audio-file, with extension: ")
            print("Decoding...")
            secret = decoding(en_file_nm)
            # Print the extracted text
            print("[INFO] Your secret message has been decoded successfully.")
            print(" ")
            print("Your secret message is: "+secret)
        
        else:
            print ("Exiting...")
            break


  
--- SELECT AN OPTION TO PERFORM AUDIO STEGANOGRAPHY ---
1.Hide a secret message in an audio file 
2.Reveal the secret message from the audio
Enter any other key to exit
Enter your key here: 1
Enter the name of your audio-file of choice, with extension: lofi.wav
Enter your secret message: Our agent has been compromised


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


[INFO] Cleanup complete
Encoding...
[INFO] YOUR MESSAGE HAS BEEN RECORDED.
[INFO] Your encoded audio is saved as 'encoded.wav'.
  
--- SELECT AN OPTION TO PERFORM AUDIO STEGANOGRAPHY ---
1.Hide a secret message in an audio file 
2.Reveal the secret message from the audio
Enter any other key to exit
Enter your key here: 2
Enter the name of your encoded audio-file, with extension: encoded.wav
Decoding...
[INFO] Your secret message has been decoded successfully.
 
Your secret message is: Our agent has been compromised
  
--- SELECT AN OPTION TO PERFORM AUDIO STEGANOGRAPHY ---
1.Hide a secret message in an audio file 
2.Reveal the secret message from the audio
Enter any other key to exit
Enter your key here: x
Exiting...
